In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

In [45]:
# 출력시 텍스트 제한 해제
pd.set_option('display.max_colwidth', None)

# 출력시 텍스트 제한
# pd.reset_option('display.max_colwidth')

In [46]:
mainPaths =[ 
    # contents_1
    './templates/contents/contents_1/귀어귀촌개념.html', 
    './templates/contents/contents_1/귀어귀촌준비절차.html', 
    './templates/contents/contents_1/어업면허허가.html', 
    './templates/contents/contents_1/어업정의.html',

    # contents_2
    './templates/contents/contents_2/귀어정책지원교육.html', 
    './templates/contents/contents_2/양식장및어선임대사업.html', 
    './templates/contents/contents_2/어촌이사혜택.html', 
    './templates/contents/contents_2/청년어촌정착지원금.html', 

    # # contents_3
    './templates/contents/contents_3/유해해양생물_종류.html', 
    './templates/contents/contents_3/유해해양생물_대응법.html', 
    './templates/contents/contents_3/이상생태현상_굴폐사.html', 
    './templates/contents/contents_3/이상생태현상_산호초백화.html', 
    './templates/contents/contents_3/이상생태현상_대응법.html', 

    # # contents_4
    './templates/contents/contents_4/IoTAI활용스마트양식.html', 
    './templates/contents/contents_4/친환경지속가능어업.html', 

    # # contents_5
    './templates/contents/contents_5/어업종류.html', 
    './templates/contents/contents_5/지역별주력어종.html', 
    './templates/contents/contents_5/금어기및금지체장.html',
    './templates/contents/contents_5/낚시금지구역.html',
    './templates/contents/contents_5/종류별낚시설명.html', 
    './templates/contents/contents_5/지역별낚시명소.html', 
    './templates/contents/contents_5/지역별낚시산업.html'
    ]

file_path = mainPaths
contentList = []

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # 총 길이
    chunk_overlap=50,   # 겹치는 길이
    separators=["\n\n", "다.", "요.", ".", "\n", " ", ""]
)



for i in file_path:
    with open(i, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    folder = os.path.basename(os.path.dirname(i)) 
    page = os.path.splitext(os.path.basename(i))[0]  
    soup = BeautifulSoup(html_content, 'html.parser')
    sections = soup.select('[name="section"]')
    titles = None
    contents = None
    title = None
    content = None
    for j in sections:
        titles = j.select('[name="content_title"]')
        contents = j.select('[name="content_text"]')
        # 둘 다 1개 이상일 때
        if len(titles) > 1 and len(contents) > 1:
            count = min(len(titles), len(contents))  # 길이 맞추기
            for k in range(count):
                title = titles[k].get_text(separator=" ",strip=True)
                content = contents[k].get_text(separator=" ", strip=True)
                chunks = splitter.split_text(content)
                for k, chunk in enumerate(chunks):
                    contentList.append([title, chunk, folder, page])

        # 하나만 있을 경우 (정확히 하나일 때만 출력)
        elif len(titles) == 1 and len(contents) == 1:
            title = titles[0].get_text(separator=" ", strip=True)
            content = contents[0].get_text(separator=" ", strip=True)
            chunks = splitter.split_text(content)
            for k, chunk in enumerate(chunks):
                contentList.append([title, chunk, folder, page])
        else:
            print(f"{f}의 제목 또는 내용이 없습니다.")
            # print(j)

df = pd.DataFrame(contentList,columns=["title", "content", "folder", "page"])
df
# len(df['title'].unique())


,title,content,folder,page
0,귀어/귀어업,어업활동을 하기 위해 타지에서부터 어촌으로 이사와 거주하는 것,contents_1,귀어귀촌개념
1,귀촌/귀어촌,귀촌/귀어촌어업활동과 관련없이 타지에서 어촌으로 이주하는 것,contents_1,귀어귀촌개념
2,귀어귀촌 준비절차,"STEP 01. 귀어를 준비하는 사람은 수산 관련기관 홈페이지에서 정보를 수집하고, 전문가와 상담을 통해 기초지식을 쌓는다. STEP 02. 귀어 희망자는 교육기관에서 어업(양식업)과 관련된 이론 및 기술 교육을 받고, 현장체험을 통해 실제 어촌 작업을 경험한다. STEP 03. 개인의 적성, 기술 수준, 자본 등을 고려하여 자신에게 적합한 업종을 신중하게 선택한다. STEP 04. 어업 또는 양식업에 직접 참여하면서 실질적인 기술을 습득한다. STEP 05. 생활 여건과 선택한 업종에 적합한 입지 조건을 바탕으로 정착할 지역을 선정한다. STEP 06. 지자체 또는 선도어가의 조언을 참고하여, 현실적으로 실행 가능한 창업계획을 수립한다. STEP 07. 창업 및 생활에 필요한 자금을 확보하고, 국가의 귀어 지원 제도를 활용한다. STEP 08. 어업 활동에 필요한 어선, 양식장, 주택 등 기반시설을 마련한",contents_1,귀어귀촌준비절차
3,귀어귀촌 준비절차,"다. STEP 08. 어업 활동에 필요한 어선, 양식장, 주택 등 기반시설을 마련한다. STEP 09. 행정 절차를 완료한 후, 수립한 계획에 따라 사업을 본격적으로 추진한다. STEP 10. 마을 행사나 회의, 공동작업 등에 적극 참여하면서 지역 주민과의 융화를 도모한다.",contents_1,귀어귀촌준비절차
4,어업면허,"어업면허 는 특정한 수면을 독점적으로 사용할 수 있도록 국가가 권리를 부여하는 제도입니다. \n 양식어업, 정치어업, 공동어업 등이 이에 해당하며 면허를 받은 자는 지정된 어장에서 타인의 출입을 배제하고 독점적으로 어업활동을 할 수 있습니다. \n 이는 출원을 통해 국가로부터 부여되는 물권적 성격의 권리이며, 일정 수면을 점용할 수 있는 권리로 등록과 함께 어업권이 발생합니다.",contents_1,어업면허허가
...,...,...,...,...
212,인천,상호명 : 피싱조이\n 홈페이지 : http://www.fishingjoy.co.kr\n 상호명 : 에프마켓\n 홈페이지 : http://www.f-market.co.kr,contents_5,지역별낚시산업
213,부산,상호명 : 유승조구산업\n 홈페이지 : http://youseunghook.co.kr/\n 상호명 : 긱스\n 홈페이지 : http://gigskorea.com\n 상호명 : 동남보트 레저산업\n 홈페이지 : http://114boat.com\n 상호명 : (주)용성\n 홈페이지 : http://www.yong-sung.co.kr,contents_5,지역별낚시산업
214,대구·울산·경상도,상호명 : 놀림낚시\n 홈페이지 : http://nollim.co.kr\n 상호명 : (주)아피스\n 홈페이지 : http://www.apis.co.kr\n 상호명 : 코러낚시\n 홈페이지 : http://www.klfishing.com/\n 상호명 : 가자낚시\n 홈페이지 : http://www.ok1717.com\n 상호명 : 현대상사\n 홈페이지 : http://www.assokorea.com\n 상호명 : 365낚시마트\n 홈페이지 : http://samyuko.kr\n 상호명 : 통영아라피싱\n 홈페이지 : http://www.kpfa.co.kr,contents_5,지역별낚시산업
215,충청도,상호명 : 한국쏘가리협회\n 홈페이지 : http://xn--o39a10a02ythks5vf5avy.com/\n 상호명 : 바다사랑\n 홈페이지 : https://www.naksi.co.kr/,contents_5,지역별낚시산업


In [19]:
# 2. 문서 객체 생성
documents = [
    Document(page_content=row['content'], metadata={"title": row['title'], "folder": row['folder'], "page": row['page']})
    for _, row in df.iterrows()
]

embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

vectorstore = FAISS.from_documents(documents=documents, embedding=embedding_model)

vectorstore.save_local("chatbot/vector_store/contents")

In [ ]:

# file_path = mainPaths
# contentList = []

# splitter = RecursiveCharacterTextSplitter(
#     chunk_size=500,     # 총 길이
#     chunk_overlap=50,   # 겹치는 길이
#     separators=["\n\n", "다.", "요.", ".", "\n", " ", ""]
# )


# with open('./templates/contents/contents_5/지역별낚시산업.html', 'r', encoding='utf-8') as f:
#     html_content = f.read()

# folder = os.path.basename(os.path.dirname(i)) 
# page = os.path.splitext(os.path.basename(i))[0]

# soup = BeautifulSoup(html_content, 'html.parser')
# sections = soup.select('[name="section"]')

# for j in sections:
#     title = j.select('[name="content_title"]')
#     ths = j.select('table thead > tr > th')
#     storeNm = ths[0].text + " : "
#     storeUrl = ths[5].text + " : "
#     thsValue = j.select('table tbody tr')

#     print(title[0].text.strip())
#     for k in thsValue:
#         storeNmTxt = k.select('td')[0].text.strip()
#         storeUrlTxt = k.select('td')[5].text.strip()
#         print(storeNm + storeNmTxt)
#         print(storeUrl + storeUrlTxt)
#     print("=======================================")   


서울
상호명 : (사)한국스포츠피싱협회
홈페이지 : http://sportfishing.co.kr/
상호명 : (사)대한스포츠낚시중앙협회
홈페이지 : https://ksfca.modoo.at/
상호명 : 한국견지낚시협회
홈페이지 : http://www.gyeonji.net/
경기도
상호명 : (주)한국낚시채널
홈페이지 : https://www.eftv.co.kr/
상호명 : (사)한국낚시협회
홈페이지 : http://ekfa.kr
상호명 : 아오맥스
홈페이지 : http://www.aomex.co.kr
상호명 : 다미끼크라프트
홈페이지 : http://www.damiki.co.kr
상호명 : 배스랜드
홈페이지 : http://www.bassland.co.kr
상호명 : 런커
홈페이지 : http://www.lunkermall.com
상호명 : (사)한국낚시업중앙회
홈페이지 : https://www.koreafca.kr/
상호명 : 한국루어낚시협회
홈페이지 : http://www.lfa.or.kr
상호명 : 한국낚시방송
홈페이지 : http://www.fishingtv.kr/
인천
상호명 : 피싱조이
홈페이지 : http://www.fishingjoy.co.kr
상호명 : 에프마켓
홈페이지 : http://www.f-market.co.kr
부산
상호명 : 유승조구산업
홈페이지 : http://youseunghook.co.kr/
상호명 : 긱스
홈페이지 : http://gigskorea.com
상호명 : 동남보트 레저산업
홈페이지 : http://114boat.com
상호명 : (주)용성
홈페이지 : http://www.yong-sung.co.kr
대구·울산·경상도
상호명 : 놀림낚시
홈페이지 : http://nollim.co.kr
상호명 : (주)아피스
홈페이지 : http://www.apis.co.kr
상호명 : 코러낚시
홈페이지 : http://www.klfishing.com/
상호명 : 가자낚시
홈페이지 : http://www.ok1717.com
상호명 :